# Amazon Personalize: 텍스트를 비정형 항목 메타데이터로 사용

Amazon Personalize를 사용하여 제공하는 추천의 연관성은 추천이 생성될 때 사용 가능한 데이터에 따라 달라집니다. Amazon Personalize는 사용자의 과거 상호 작용, 항목의 속성 및 사용자의 메타데이터를 사용하여 각 사용자에게 가장 적합한 항목을 찾습니다. Amazon Personalize에 필요한 주 데이터는 사용자-항목 상호 작용입니다. 제품 클릭, 기사 읽기, 동영상 시청, 제품 구매 등, 사용자가 카탈로그의 항목에 대해 수행하는 상호 작용은 과거에 연관성이 있다고 판단한 항목을 나타내는 중요한 신호입니다. 메타데이터라고도 하는 항목 및 사용자 속성을 포함하면, 추천의 연관성을 높일 수 있습니다. 특히 사용자가 연관성이 있다고 판단한 항목과 유사한 새 항목의 경우 더욱 그렇습니다. 하지만 항목의 범주, 스타일 또는 장르와 같은 정형 메타데이터는 사용하기가 쉽지 않거나 서사적 기술에 있는 정보를 완전하게 제공하지 않을 수 있습니다. 이제 Amazon Personalize를 사용하면 제품 설명, 비디오 스크립트 또는 기사 텍스트와 같은 비정형 메타데이터를 다른 항목 속성과 함께 추가할 수 있습니다. Amazon Personalize는 자연어 처리(NLP) 모델을 호스팅하고 관리하고 자동으로 사용하여 텍스트를 처리하고, Amazon Personalize 솔루션의 성능을 개선하는 데 활용합니다.

이 노트북에서는 추천의 연관성을 개선하기 위해 제품 설명 형식의 텍스트를 비정형 항목 메타데이터로 포함하는 방법을 보여줍니다.

Amazon Prime Pantry 범주의 Amazon 리뷰 데이터가 상호 작용 및 항목 데이터 세트로 사용됩니다.

항목 데이터 세트에 텍스트를 포함할 때는 다음 모범 사례를 고려하세요.
- 각 항목에 대해 간결하고, 연관성이 있으며, 유용한 정보를 제공하는 것으로 편집상 검증된 텍스트이며, 텍스트의 앞부분에 언급된 가장 연관성이 높은 정보는 관련성이 낮거나 일관성이 있는 사용자 생성 콘텐츠보다 선호됩니다.
- 텍스트 열이 희박하게 채워지면 항목 데이터 세트에 텍스트를 포함할 때의 긍정적인 영향이 줄어듭니다.
- 항목 데이터 세트에 추가하기 전에 마크업 및 불필요한 공백 형식의 텍스트를 모두 지웁니다.
- 텍스트 필드에서는 현재 영어만 지원됩니다.
- 텍스트 필드는 현재 사용자 개인화 및 개인별 순위 레시피에만 고려됩니다.

항목 설명이 있는 데이터와 없는 데이터를 포함하는 두 개의 데이터 세트 그룹이 생성되므로, 각각 별도의 모델을 훈련하고 오프라인 및 온라인 결과를 비교할 수 있습니다.

In [1]:
import pandas as pd
import json
import numpy as np
from datetime import datetime
import boto3
import time
from time import sleep
from lxml import html

## 데이터 세트를 로드하고 살펴보기

Prime Pantry 리뷰 데이터 세트를 로드하는 것으로 시작하겠습니다. 데이터 파일에 액세스하려면 다음 양식을 작성해야 합니다.

http://deepyeti.ucsd.edu/jianmo/amazon/index.html

Citation:
> Justifying recommendations using distantly-labeled reviews and fined-grained aspects  
> Jianmo Ni, Jiacheng Li, Julian McAuley  
> Empirical Methods in Natural Language Processing (EMNLP), 2019 [pdf](http://cseweb.ucsd.edu/~jmcauley/pdfs/emnlp19a.pdf)

In [2]:
data_dir = 'raw_data'
!mkdir $data_dir

!cd $data_dir && \
    wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFiles/Prime_Pantry.json.gz && \
    wget http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_Prime_Pantry.json.gz

mkdir: cannot create directory ‘raw_data’: File exists
--2021-07-13 22:06:52--  http://deepyeti.ucsd.edu/jianmo/amazon/categoryFiles/Prime_Pantry.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 45435146 (43M) [application/octet-stream]
Saving to: ‘Prime_Pantry.json.gz’

Prime_Pantry.json.g 100%[===================>]  43.33M  12.3MB/s    in 4.8s    

2021-07-13 22:06:57 (9.01 MB/s) - ‘Prime_Pantry.json.gz’ saved [45435146/45435146]

--2021-07-13 22:06:57--  http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_Prime_Pantry.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5281662 (5.0M) [application/octet-stream]
Saving to: ‘meta_Prime_Pantry.json.gz’

meta_Prime_Pa

### 리뷰 데이터를 로드하고 살펴보기

먼저 Prime Pantry 제품에 대한 리뷰 데이터 세트를 로드하고 몇 가지 명령을 실행하여 작업 내용을 살펴보겠습니다.

In [3]:
pantry_df = pd.read_json(data_dir + '/Prime_Pantry.json.gz', lines=True, compression='infer')
pantry_df.head()

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,image,style
0,5,True,"12 14, 2014",A1NKJW0TNRVS7O,B0000DIWNZ,Tamara M.,Good clinging,Clings well,1418515200,NaN,NaN,NaN
1,4,True,"11 20, 2014",A2L6X37E8TFTCC,B0000DIWNZ,Amazon Customer,Fantastic buy and a good plastic wrap. Even t...,Saran could use more Plus to Cling better.,1416441600,NaN,NaN,NaN
2,4,True,"10 11, 2014",A2WPR4W6V48121,B0000DIWNZ,noname,ok,Four Stars,1412985600,NaN,NaN,NaN
3,3,False,"09 1, 2014",A27EE7X7L29UMU,B0000DIWNZ,ZapNZs,Saran Cling Plus is kind of like most of the C...,"The wrap is fantastic, but the dispensing, cut...",1409529600,4,NaN,NaN
4,4,True,"08 10, 2014",A1OWT4YZGB5GV9,B0000DIWNZ,Amy Rogers,This is my go to plastic wrap so there isn't m...,has been doing it's job for years,1407628800,NaN,NaN,NaN


In [4]:
pantry_df.shape

(471614, 12)

이 출력으로부터 무엇을 알 수 있을까요? 471,000건 이상의 리뷰와 12개의 데이터 열이 있습니다. `asin` 열은 고유한 항목 식별자이고, `reviewerID`는 고유한 사용자 식별자이며, `unixReviewTime`은 리뷰의 타임스탬프이고, `overall`은 1~5의 척도로 리뷰의 긍정성을 나타냅니다. 이 파일을 Personalize를 위한 상호 작용 데이터 세트의 기반으로 사용합니다. 

### 상호 작용 데이터 세트 구축 및 저장

이제 포함할 행의 범위를 좁혀 상호 작용 데이터 세트를 구축해 보겠습니다. 첫 단계는 긍정적인 리뷰만 격리하는 것입니다. 이를 위해 전반적인 평점이 4점 이상인 리뷰가 긍정적인 리뷰라고 가정합니다. 3점 이하의 리뷰는 보통이거나 부정적인 리뷰입니다.

In [5]:
positive_reviews_df = pantry_df[pantry_df['overall'] > 3]
positive_reviews_df.shape

(387692, 12)

이 예에는 387,000개의 긍정적인 리뷰가 있습니다. Personalize에서 모델을 훈련하기에 충분합니다.

이제 필요한 열만 포함하도록 데이터 세트를 줄이고 캡처할 이벤트 유형을 나타내는 `EVENT_TYPE` 열을 추가하겠습니다. `eventType`은 [PutEvents](https://docs.aws.amazon.com/personalize/latest/dg/API_UBS_PutEvents.html) API의 필수 필드이므로, 지금 `EVENT_TYPE` 열을 추가하면 나중에 테스트 실시간 이벤트를 손쉽게 탐색할 수 있습니다.

In [6]:
positive_reviews_df = positive_reviews_df[['reviewerID', 'asin', 'unixReviewTime', 'overall']]
positive_reviews_df['EVENT_TYPE']='reviewed'

positive_reviews_df.head()

,reviewerID,asin,unixReviewTime,overall,EVENT_TYPE
0,A1NKJW0TNRVS7O,B0000DIWNZ,1418515200,5,reviewed
1,A2L6X37E8TFTCC,B0000DIWNZ,1416441600,4,reviewed
2,A2WPR4W6V48121,B0000DIWNZ,1412985600,4,reviewed
4,A1OWT4YZGB5GV9,B0000DIWNZ,1407628800,4,reviewed
5,A1GN2ADKF1IE7K,B0000DIWNZ,1405296000,5,reviewed


마지막으로 `unixReviewTime` 열 값의 온전성을 확인해야 합니다. Personalize는 각 상호 작용의 날짜 및 시간을 기준으로 시퀀스 모델을 구축하므로, 각 상호 작용의 타임스탬프가 올바르게 해석되도록 타임스탬프를 요구되는 형식으로 표현하는 것이 중요합니다.

`unixReviewTime` 열의 값을 선택하고 사용자가 읽을 수 있는 날짜로 구문 분석하여 타당한지 확인합니다.

In [7]:
time_stamp = positive_reviews_df.iloc[50]['unixReviewTime']
print(time_stamp)
print(datetime.utcfromtimestamp(time_stamp).strftime('%Y-%m-%d %H:%M:%S'))

1321488000
2011-11-17 00:00:00


타임스탬프 값이 괜찮아 보입니다. 데이터 세트에 대한 최종 요약 정보를 확인해 보겠습니다.

In [8]:
positive_reviews_df.describe(include='all')

,reviewerID,asin,unixReviewTime,overall,EVENT_TYPE
count,387692,387692,3.876920e+05,387692.000000,387692
unique,202254,10584,NaN,NaN,1
top,A35Q0RBM3YNQNF,B00XA9DADC,NaN,NaN,reviewed
freq,176,5288,NaN,NaN,387692
mean,NaN,NaN,1.468847e+09,4.847227,NaN
std,NaN,NaN,4.314975e+07,0.359769,NaN
min,NaN,NaN,1.073693e+09,4.000000,NaN
25%,NaN,NaN,1.447200e+09,5.000000,NaN
50%,NaN,NaN,1.474718e+09,5.000000,NaN
75%,NaN,NaN,1.498435e+09,5.000000,NaN


10,000개의 고유 제품 전체에 걸쳐 202,000명의 개별 리뷰어/사용자가 작성한 387,000건의 리뷰가 있습니다. 이 데이터가 상호 작용 데이터 세트의 기초가 됩니다.

단, 이 정보를 상호 작용 데이터 세트로 사용하려면 먼저 열 이름을 Personalize가 사용하는 이름과 일치하도록 변경해야 합니다.

In [9]:
positive_reviews_df.rename(columns = {'reviewerID':'USER_ID', 'asin':'ITEM_ID', 
                              'unixReviewTime':'TIMESTAMP', 'overall': 'EVENT_VALUE'}, inplace = True)
positive_reviews_df

,USER_ID,ITEM_ID,TIMESTAMP,EVENT_VALUE,EVENT_TYPE
0,A1NKJW0TNRVS7O,B0000DIWNZ,1418515200,5,reviewed
1,A2L6X37E8TFTCC,B0000DIWNZ,1416441600,4,reviewed
2,A2WPR4W6V48121,B0000DIWNZ,1412985600,4,reviewed
4,A1OWT4YZGB5GV9,B0000DIWNZ,1407628800,4,reviewed
5,A1GN2ADKF1IE7K,B0000DIWNZ,1405296000,5,reviewed
...,...,...,...,...,...
471609,A19GSVHXVT5NNF,B01HI8JVI8,1494892800,5,reviewed
471610,ABSCTKLX9F9IU,B01HI8JVI8,1493769600,5,reviewed
471611,A2R33RCWKDHZ3L,B01HI8JVI8,1492646400,5,reviewed
471612,A2INGHYEXZDHMC,B01HI8JVI8,1492560000,5,reviewed


마지막으로, 긍정적인 리뷰 데이터 프레임을 CSV로 저장하겠습니다. 이 노트북의 뒷부분에서 이 CSV를 Personalize에 업로드합니다.

In [10]:
interactions_filename = "interactions.csv"
positive_reviews_df.to_csv(interactions_filename, index=False, float_format='%.0f')

### 항목 메타데이터를 로드하고 살펴보기

이제 상호 작용 데이터 세트가 준비되었으므로, 항목 데이터 세트를 만들겠습니다. 이 단계에서는 모델에 포함할 비정형 텍스트 값을 찾습니다.

리뷰 데이터 세트와 마찬가지로, Prime Pantry 항목 메타데이터 파일도 JSON으로 표현됩니다. 이 파일의 중첩되는 특성으로 인해 데이터를 필요한 형식으로 변경하는 데 몇 가지 어려움이 있습니다.

먼저 메타데이터 파일을 데이터 프레임에 로드하고 데이터를 살펴보겠습니다.

In [11]:
pantry_meta_df = pd.read_json('raw_data/meta_Prime_Pantry.json.gz', lines=True, compression='infer')
pantry_meta_df

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes
0,[],,[Sink your sweet tooth into MILK DUDS Candya d...,,"HERSHEY'S Milk Duds Candy, 5 Ounce(Halloween C...","[B019KE37WO, B007NQSWEU]",,Milk Duds,[],[],[],"{'ASIN: ': 'B00005BPJO', 'Item model number:':...","<img src=""https://m.media-amazon.com/images/G/...",,NaT,$5.00,B00005BPJO,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
1,[],,[Sink your sweet tooth into MILK DUDS Candya d...,,"HERSHEY'S Milk Duds Candy, 5 Ounce(Halloween C...","[B019KE37WO, B007NQSWEU]",,Milk Duds,[],[],[],"{'ASIN: ': 'B00005BPJO', 'Item model number:':...","<img src=""https://m.media-amazon.com/images/G/...",,NaT,$5.00,B00005BPJO,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
2,[],,[A perfect Lentil soup starts with Goya Lentil...,,"Goya Dry Lentils, 16 oz","[B003SI144W, B000VDRKEK]",,Goya,[],[],"[B074MFVZG7, B079PTH69L, B000VDRKEK, B074M9T81...",{'ASIN: ': 'B0000DIF38'},"<img src=""https://images-na.ssl-images-amazon....",,NaT,,B0000DIF38,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
3,[],,[Saran Premium Wrap is an extra tough yet easy...,,"Saran Premium Plastic Wrap, 100 Sq Ft","[B01MY5FHT6, B000PYF8VM, B000SRMDFA, B07CX6LN8...",,Saran,[],[],"[B077QLSLRQ, B00JPKW1RQ, B000FE2IK6, B00XUJHJ9...",{'Domestic Shipping: ': 'This item can only be...,"<img src=""https://images-na.ssl-images-amazon....",,NaT,,B0000DIWNI,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
4,[],,[200 sq ft (285 ft x 11-3/4 in x 18.6 m2). Eas...,,"Saran Cling Plus Plastic Wrap, 200 Sq Ft",[],,Saran,[],[],[B0014CZ0TE],{'Domestic Shipping: ': 'This item can only be...,"<img src=""https://images-na.ssl-images-amazon....",,NaT,,B0000DIWNZ,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10808,[],,[These bars are where our journey started and ...,,"KIND Bars, Caramel Almond &amp; Sea Salt, Glut...",[],,KIND,[],"26,259 in Grocery & Gourmet Food (","[B00JQQAN60, B00JQQAWSY, B0111K7V54, B0111K8L9...","{'ASIN: ': 'B01HI76312', 'Item model number:':...","<img src=""https://images-na.ssl-images-amazon....",,NaT,$3.98,B01HI76312,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
10809,[],,[These bars are where our journey started and ...,,"KIND Bars, Maple Glazed Pecan &amp; Sea Salt, ...",[],,KIND,[],"16,822 in Grocery & Gourmet Food (","[B0111K97JC, B00JQQAN60, B0111K8L9Y, B01HI7631...",{'ASIN: ': 'B01HI76790'},"<img src=""https://images-na.ssl-images-amazon....",,NaT,$5.81,B01HI76790,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
10810,[],,[These bars are where our journey started and ...,,"KIND Bars, Dark Chocolate Almond &amp; Coconut...",[],,KIND,[],"107,057 in Grocery & Gourmet Food (","[B0111K7V54, B01HI76312, B00JQQAL0S, B0111K97J...",{'ASIN: ': 'B01HI76SA8'},"<img src=""https://images-na.ssl-images-amazon....",,NaT,$4.98,B01HI76SA8,[],[]
10811,[],,[These bars are where our journey started and ...,,"KIND Bars, Honey Roasted Nuts &amp; Sea Salt, ...",[],,KIND,[],"24,648 in Grocery & Gourmet Food (","[B00JQQAN60, B0111K7V54, B01HI76312, B0111K97J...",{'ASIN: ': 'B01HI76XS0'},"<img src=""https://images-na.ssl-images-amazon....",,NaT,$5.81,B01HI76XS0,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...


In [12]:
pantry_meta_df.describe()

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes
count,10813,10813,10813,10813,10813,10813,10813,10813,10813,10813,10813,10813,10813,10813,0,10813,10813,10813,10813
unique,1,1,9409,1,10782,3957,1,1960,763,4828,5940,10786,4,1,0,1482,10812,8940,8940
top,[],,[],,"Infants' Motrin Concentrated Drops, Fever Redu...",[],,L'Oreal Paris,[],[],[],{},"<img src=""https://images-na.ssl-images-amazon....",,NaN,,B00005BPJO,[],[]
freq,10813,10813,98,10813,2,6754,10813,171,9777,5937,4835,24,10621,10813,NaN,4063,2,1781,1781


이 정보로부터 무엇을 알 수 있을까요? 첫째, 메타데이터 파일에 10,000 이상의 제품이 있습니다. 대부분의 열은 특성(이미지 URL, `details`, `also_viewed`, `also_buy` 등)과 연관성이 없거나 대부분 공백/희소 데이터(`category`, `fit`, `tech1` 등)이므로 Personalize에는 별 가치가 없습니다. 복제본이 하나 있는 것처럼 보이지만 `asin` 열은 각 항목의 고유한 식별자이며. `brand`와 `price`는 유용할 것 같습니다. `description` 열은 비정형 텍스트에 사용됩니다.

하지만 항목 데이터 세트에 사용할 필드를 정리하고 형식을 다시 설정해야 합니다. 예를 들어 `price` 필드는 숫자 이외의 형식으로 된 통화 값(문자열)이며, `description` 필드는 원본 JSON 파일에서 값이 표현되고 구문 분석되는 방식으로 인해 문자열 배열로 로드되었습니다. 마지막으로, `description` 값에는 제거해야 할 HTML 마크업도 포함되어 있습니다.

먼저 항목 데이터 세트에 필요한 열만 사용하여 데이터 프레임을 생성하겠습니다.

In [13]:
items_df = pantry_meta_df.copy()
items_df = items_df[['asin', 'brand', 'price', 'description']]
items_df.head(10)

,asin,brand,price,description
0,B00005BPJO,Milk Duds,$5.00,[Sink your sweet tooth into MILK DUDS Candya d...
1,B00005BPJO,Milk Duds,$5.00,[Sink your sweet tooth into MILK DUDS Candya d...
2,B0000DIF38,Goya,,[A perfect Lentil soup starts with Goya Lentil...
3,B0000DIWNI,Saran,,[Saran Premium Wrap is an extra tough yet easy...
4,B0000DIWNZ,Saran,,[200 sq ft (285 ft x 11-3/4 in x 18.6 m2). Eas...
5,B0000GH6UG,Ibarra,,"[Ibarra Chocolate, 19 Oz, , ]"
6,B0000KC2BK,Knorr,$3.09,[Knorr Granulated Chicken Flavor Bouillon is a...
7,B0001E1IN8,Castillo,,[Red chili habanero sauces. They are present t...
8,B00032E8XK,Chicken of the Sea,$1.48,[Chicken of the Sea Solid White Albacore Tuna ...
9,B0005XMTHE,Smucker's,$2.29,"[Helps build muscles with bcaa's amino acids, ..."


다음으로, `asin` 열 값을 기준으로 중복되는 행을 삭제하겠습니다. 위의 `describe()` 출력으로 볼 때, 중복되는 항목은 하나뿐입니다.

In [14]:
items_df = items_df.drop_duplicates(subset=['asin'], keep='last')
items_df.shape

(10812, 4)

다음으로, 다시 형식을 지정하고 `description` 열 값을 정리합니다. 위에서 보듯이 `description`은 현재 문자열 배열로 표현됩니다. JSON 파일에서 이렇게 표현되기 때문입니다. 이 배열을 하나의 문자열로 정리하고, 각각에서 모든 HTML 마크업을 제거해야 합니다.

먼저 `description`을 정리(추천 제품의 제목을 표시하는 경우 나중에 원본 데이터 세트의 `title` 열도 정리)하는 데 사용할 두 가지 유틸리티 함수를 만듭니다.

In [15]:
# Strips and cleans a value of HTML markup and whitespace.
def clean_markup(value):
    s = str(value).strip()
    if s != '':
        s = str(html.fromstring(s).text_content())
        s = ' '.join(s.split())
                
    return s.strip()

# Cleans and reformats the description column value for a dataframe row.
def clean_and_reformat_description(row):
    s = ''
    for el in row['description']:
        el = clean_markup(el)
        if el != '':
            s += ' ' + el
                
    return s.strip()

In [16]:
items_df['description'] = items_df.apply(clean_and_reformat_description, axis=1)
items_df

,asin,brand,price,description
1,B00005BPJO,Milk Duds,$5.00,Sink your sweet tooth into MILK DUDS Candya de...
2,B0000DIF38,Goya,,A perfect Lentil soup starts with Goya Lentils...
3,B0000DIWNI,Saran,,Saran Premium Wrap is an extra tough yet easy ...
4,B0000DIWNZ,Saran,,200 sq ft (285 ft x 11-3/4 in x 18.6 m2). Easy...
5,B0000GH6UG,Ibarra,,"Ibarra Chocolate, 19 Oz"
...,...,...,...,...
10808,B01HI76312,KIND,$3.98,These bars are where our journey started and i...
10809,B01HI76790,KIND,$5.81,These bars are where our journey started and i...
10810,B01HI76SA8,KIND,$4.98,These bars are where our journey started and i...
10811,B01HI76XS0,KIND,$5.81,These bars are where our journey started and i...


다음으로, `price` 열을 살펴보고 이 열의 형식을 문자열에서 실수로 변경해 보겠습니다.

In [17]:
items_df['price'].value_counts()

          4063
$2.99      114
$3.99      113
$4.99      103
$5.99       87
          ... 
$20.42       1
$32.32       1
$1.52        1
$27.89       1
$39.10       1
Name: price, Length: 1482, dtype: int64

다음 셀은 빈 값/숫자 이외의 값을 `np.nan`으로 변환하며, 다른 값에서는 `$` 통화 기호를 모두 제거합니다. 이렇게 하면 강제로 형식을 실수로 변경할 수 있습니다.

In [18]:
def convert_price(row):
    v = str(row['price']).strip().replace('$', '')
    if v == '' or not v.lstrip('-').replace('.', '').isdigit():
        return np.nan
    return v

items_df['price'] = items_df.apply(convert_price, axis=1)
items_df

,asin,brand,price,description
1,B00005BPJO,Milk Duds,5.00,Sink your sweet tooth into MILK DUDS Candya de...
2,B0000DIF38,Goya,NaN,A perfect Lentil soup starts with Goya Lentils...
3,B0000DIWNI,Saran,NaN,Saran Premium Wrap is an extra tough yet easy ...
4,B0000DIWNZ,Saran,NaN,200 sq ft (285 ft x 11-3/4 in x 18.6 m2). Easy...
5,B0000GH6UG,Ibarra,NaN,"Ibarra Chocolate, 19 Oz"
...,...,...,...,...
10808,B01HI76312,KIND,3.98,These bars are where our journey started and i...
10809,B01HI76790,KIND,5.81,These bars are where our journey started and i...
10810,B01HI76SA8,KIND,4.98,These bars are where our journey started and i...
10811,B01HI76XS0,KIND,5.81,These bars are where our journey started and i...


In [19]:
items_df['price'].value_counts()

2.99     114
3.99     113
4.99     103
5.99      87
2.98      76
        ... 
39.10      1
1.84       1
22.95      1
12.17      1
11.09      1
Name: price, Length: 1480, dtype: int64

In [20]:
items_df['price'] = items_df['price'].astype(float)

그런 다음 Personalize에 필요한 이름 및 대문자 이름 형식에 맞게 열 이름을 변경합니다.

In [21]:
items_df.rename(columns = {'asin':'ITEM_ID', 'brand':'BRAND', 
                              'price':'PRICE', 'description': 'DESCRIPTION'}, inplace = True)
items_df.head(10)

,ITEM_ID,BRAND,PRICE,DESCRIPTION
1,B00005BPJO,Milk Duds,5.00,Sink your sweet tooth into MILK DUDS Candya de...
2,B0000DIF38,Goya,NaN,A perfect Lentil soup starts with Goya Lentils...
3,B0000DIWNI,Saran,NaN,Saran Premium Wrap is an extra tough yet easy ...
4,B0000DIWNZ,Saran,NaN,200 sq ft (285 ft x 11-3/4 in x 18.6 m2). Easy...
5,B0000GH6UG,Ibarra,NaN,"Ibarra Chocolate, 19 Oz"
6,B0000KC2BK,Knorr,3.09,Knorr Granulated Chicken Flavor Bouillon is a ...
7,B0001E1IN8,Castillo,NaN,Red chili habanero sauces. They are present to...
8,B00032E8XK,Chicken of the Sea,1.48,Chicken of the Sea Solid White Albacore Tuna i...
9,B0005XMTHE,Smucker's,2.29,"Helps build muscles with bcaa's amino acids, i..."
10,B0005XNE6E,Snapple,1.99,"At Snapple, we believe lifes a peach. Weve bee..."


2개의 항목으로 이루어진 CSV를 생성합니다. 그중 하나에는 설명 열이 있고 다른 하나에는 없습니다. 이러한 각 항목을 사용하여 동일한 레시피로 별도의 모델을 훈련하여 오프라인 지표를 비교하고 추천에 대한 온라인 조사를 수행할 수 있습니다.

In [22]:
items_with_desc_filename = "items-with-desc.csv"
items_df.to_csv(items_with_desc_filename, index=False, float_format='%.2f')

설명 열이 제거된 다른 항목 CSV입니다.

In [23]:
items_without_desc_df = items_df[['ITEM_ID', 'BRAND', 'PRICE']]
items_without_desc_df.head()

,ITEM_ID,BRAND,PRICE
1,B00005BPJO,Milk Duds,5.0
2,B0000DIF38,Goya,NaN
3,B0000DIWNI,Saran,NaN
4,B0000DIWNZ,Saran,NaN
5,B0000GH6UG,Ibarra,NaN


In [24]:
items_without_desc_filename = "items-without-desc.csv"
items_without_desc_df.to_csv(items_without_desc_filename, index=False, float_format='%.2f')

## 데이터 세트 그룹 생성 및 데이터 세트 업로드

이제 필요한 데이터 세트를 구축했으므로, 데이터 세트 가져오기 작업을 사용하여 Personalize에 업로드해야 합니다. CSV를 업로드하려면 먼저 설명이 있고 없는 두 데이터 세트를 보류할 데이터 세트 그룹을 생성하고, 데이터 세트의 스키마를 생성하고, 데이터 세트를 생성해야 합니다.

먼저 Personalize와 상호 작용해야 하는 SDK 클라이언트를 만듭니다.

In [25]:
personalize = boto3.client('personalize')

### 데이터 세트 그룹 생성

2개의 데이터 세트 그룹을 만들어 보겠습니다.

In [26]:
create_dataset_group_response = personalize.create_dataset_group(
    name = "amazon-pantry-without-desc"
)

dataset_group_without_desc_arn = create_dataset_group_response['datasetGroupArn']
print(json.dumps(create_dataset_group_response, indent=2))

{
  "datasetGroupArn": "arn:aws:personalize:us-east-1:224124347618:dataset-group/amazon-pantry-without-desc",
  "ResponseMetadata": {
    "RequestId": "20bd153c-ebd0-432d-9ef3-522a0d2fd8d4",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 13 Jul 2021 22:08:18 GMT",
      "x-amzn-requestid": "20bd153c-ebd0-432d-9ef3-522a0d2fd8d4",
      "content-length": "105",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [27]:
create_dataset_group_response = personalize.create_dataset_group(
    name = "amazon-pantry-with-desc"
)

dataset_group_with_desc_arn = create_dataset_group_response['datasetGroupArn']
print(json.dumps(create_dataset_group_response, indent=2))

{
  "datasetGroupArn": "arn:aws:personalize:us-east-1:224124347618:dataset-group/amazon-pantry-with-desc",
  "ResponseMetadata": {
    "RequestId": "9cec53c8-28a3-40e5-bf0e-6f87a03113f7",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 13 Jul 2021 22:08:18 GMT",
      "x-amzn-requestid": "9cec53c8-28a3-40e5-bf0e-6f87a03113f7",
      "content-length": "102",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


데이터 세트 그룹이 생성되는 데 몇 초 정도 걸릴 수 있으므로, 두 그룹 모두 활성 상태가 될 때까지 기다려 보겠습니다.

In [28]:
in_progress_dataset_group_arns = [ dataset_group_without_desc_arn, dataset_group_with_desc_arn ]

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    for dataset_group_arn in in_progress_dataset_group_arns:
        describe_dataset_group_response = personalize.describe_dataset_group(
            datasetGroupArn = dataset_group_arn
        )
        status = describe_dataset_group_response["datasetGroup"]["status"]
        if status == "ACTIVE":
            print("Dataset group create succeeded for {}".format(dataset_group_arn))
            in_progress_dataset_group_arns.remove(dataset_group_arn)
        elif status == "CREATE FAILED":
            print("Create failed for {}".format(dataset_group_arn))
            in_progress_dataset_group_arns.remove(dataset_group_arn)

    if len(in_progress_dataset_group_arns) <= 0:
        break
    else:
        print("At least one dataset group create is still in progress")
                
    time.sleep(10)

At least one dataset group create is still in progress
Dataset group create succeeded for arn:aws:personalize:us-east-1:224124347618:dataset-group/amazon-pantry-without-desc
At least one dataset group create is still in progress
Dataset group create succeeded for arn:aws:personalize:us-east-1:224124347618:dataset-group/amazon-pantry-with-desc


### 상호 작용 데이터 세트 스키마 및 데이터 세트 생성

상호 작용 데이터 세트는 두 데이터 세트 그룹에서 동일하므로, 상호 작용 데이터 세트 유형에 대한 단일 스키마를 생성하고 두 데이터 세트 그룹 간에 공유합니다. 스키마는 AWS 계정에 전역적으로 적용되고 데이터 세트 그룹에만 국한되지 않으므로 공유할 수 있습니다.

In [29]:
interactions_schema = schema = {
    "type": "record",
    "name": "Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "TIMESTAMP",
            "type": "long"
        },
        {
            "name": "EVENT_VALUE",
            "type": "float"
        },
        {
            "name": "EVENT_TYPE",
            "type": "string"
        }
    ],
    "version": "1.0"
}
            
create_schema_response = personalize.create_schema(
    name = "amazon-pantry-interactions",
    schema = json.dumps(interactions_schema)
)

interaction_schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

{
  "schemaArn": "arn:aws:personalize:us-east-1:224124347618:schema/amazon-pantry-interactions",
  "ResponseMetadata": {
    "RequestId": "6be5e019-0c8e-487b-a158-6f089f9e79ca",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 13 Jul 2021 22:08:40 GMT",
      "x-amzn-requestid": "6be5e019-0c8e-487b-a158-6f089f9e79ca",
      "content-length": "92",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


다음으로, 방금 만든 스키마를 지정하여, 두 데이터 세트 그룹에 상호 작용 데이터 세트를 만듭니다.

In [30]:
dataset_type = "INTERACTIONS"
create_dataset_response = personalize.create_dataset(
    name = "amazon-pantry-without-desc-ints",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_without_desc_arn,
    schemaArn = interaction_schema_arn
)

interactions_dataset_without_desc_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

{
  "datasetArn": "arn:aws:personalize:us-east-1:224124347618:dataset/amazon-pantry-without-desc/INTERACTIONS",
  "ResponseMetadata": {
    "RequestId": "ea1020bb-a7fa-4a64-a29e-f07e8e0f9ae9",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 13 Jul 2021 22:08:41 GMT",
      "x-amzn-requestid": "ea1020bb-a7fa-4a64-a29e-f07e8e0f9ae9",
      "content-length": "107",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [31]:
create_dataset_response = personalize.create_dataset(
    name = "amazon-pantry-with-desc-ints",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_with_desc_arn,
    schemaArn = interaction_schema_arn
)

interactions_dataset_with_desc_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

{
  "datasetArn": "arn:aws:personalize:us-east-1:224124347618:dataset/amazon-pantry-with-desc/INTERACTIONS",
  "ResponseMetadata": {
    "RequestId": "614257cc-6344-408e-8e15-89d4f81ae927",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 13 Jul 2021 22:08:41 GMT",
      "x-amzn-requestid": "614257cc-6344-408e-8e15-89d4f81ae927",
      "content-length": "104",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### S3에 상호 작용 CSV 스테이징

앞에 생성한 상호 작용 CSV를 방금 만든 Personalize 데이터 세트에 업로드하려면 먼저 CSV를 S3 버킷에 스테이징해야 합니다.

S3 버킷을 만들어 상호 작용 CSV 파일을 그 버킷에 복사해 보겠습니다.

In [32]:
# Determine the current S3 region where this notebook is being hosted in SageMaker.
with open('/opt/ml/metadata/resource-metadata.json') as notebook_info:
    data = json.load(notebook_info)
    resource_arn = data['ResourceArn']
    region = resource_arn.split(':')[3]
print(region)

us-east-1


In [33]:
s3 = boto3.client('s3')
account_id = boto3.client('sts').get_caller_identity().get('Account')
bucket_name = account_id + "-" + region + "-" + "amazon-pantry-personalize-text"
print(bucket_name)
if region == "us-east-1":
    s3.create_bucket(Bucket=bucket_name)
else:
    s3.create_bucket(
        Bucket=bucket_name,
        CreateBucketConfiguration={'LocationConstraint': region}
    )

224124347618-us-east-1-amazon-pantry-personalize-text


#### S3에 상호 작용 CSV 업로드

In [34]:
boto3.Session().resource('s3').Bucket(bucket_name).Object(interactions_filename).upload_file(interactions_filename)

### S3 버킷 정책 및 IAM 역할 생성

데이터 세트 가져오기 작업을 Personalize에 제출하려면, 먼저 버킷에 대한 Personalize 액세스 권한을 부여하는 버킷 정책 및 IAM 역할을 생성해야 합니다.

In [35]:
policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:*Object",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket_name),
                "arn:aws:s3:::{}/*".format(bucket_name)
            ]
        }
    ]
}

s3.put_bucket_policy(Bucket=bucket_name, Policy=json.dumps(policy))

{'ResponseMetadata': {'RequestId': 'SBN10P9R7H7ST5RK',
  'HostId': 'DD8fYEx27yBq6/rB7o9lMvkdCLOHOewN05NSq73g30jeFBdouLj5D+fWSnIZHvDuAKdCKEo7w3k=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': 'DD8fYEx27yBq6/rB7o9lMvkdCLOHOewN05NSq73g30jeFBdouLj5D+fWSnIZHvDuAKdCKEo7w3k=',
   'x-amz-request-id': 'SBN10P9R7H7ST5RK',
   'date': 'Tue, 13 Jul 2021 22:10:59 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

In [36]:
iam = boto3.client("iam")

role_name = "PersonalizeRoleAmazonPantry"
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}

create_role_response = iam.create_role(
    RoleName = role_name,
    AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
)

# AmazonPersonalizeFullAccess provides access to any S3 bucket with a name that includes "personalize" or "Personalize" 
# if you would like to use a bucket with a different name, please consider creating and attaching a new policy
# that provides read access to your bucket or attaching the AmazonS3ReadOnlyAccess policy to the role
policy_arn = "arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess"
iam.attach_role_policy(
    RoleName = role_name,
    PolicyArn = policy_arn
)

# Now add S3 support
iam.attach_role_policy(
    PolicyArn='arn:aws:iam::aws:policy/AmazonS3FullAccess',
    RoleName=role_name
)
time.sleep(20) # wait for a minute to allow IAM role policy attachment to propagate

role_arn = create_role_response["Role"]["Arn"]
print(role_arn)

arn:aws:iam::224124347618:role/PersonalizeRoleAmazonPantry


### 각 데이터 세트 그룹의 상호 작용 데이터 세트 가져오기

이제 S3 버킷에 있는 스테이징된 상호 작용 CSV를 앞서 각 데이터 세트 그룹에 생성한 Personalize 데이터 세트로 가져올 준비가 되었습니다. 2개의 가져오기 작업을 모두 제출하고 완료되기를 기다리겠습니다.

In [37]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "amazon-pantry-without-desc-ints-import",
    datasetArn = interactions_dataset_without_desc_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, interactions_filename)
    },
    roleArn = role_arn
)

dataset_import_job_without_ints_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

{
  "datasetImportJobArn": "arn:aws:personalize:us-east-1:224124347618:dataset-import-job/amazon-pantry-without-desc-ints-import",
  "ResponseMetadata": {
    "RequestId": "84c4d71d-fe71-4ee7-bccf-4f8ed8b1e549",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 13 Jul 2021 22:12:08 GMT",
      "x-amzn-requestid": "84c4d71d-fe71-4ee7-bccf-4f8ed8b1e549",
      "content-length": "126",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [38]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "amazon-pantry-with-desc-ints-import",
    datasetArn = interactions_dataset_with_desc_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, interactions_filename)
    },
    roleArn = role_arn
)

dataset_import_job_with_ints_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

{
  "datasetImportJobArn": "arn:aws:personalize:us-east-1:224124347618:dataset-import-job/amazon-pantry-with-desc-ints-import",
  "ResponseMetadata": {
    "RequestId": "eae39716-264b-48e8-bfb4-93f569c7a904",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 13 Jul 2021 22:12:09 GMT",
      "x-amzn-requestid": "eae39716-264b-48e8-bfb4-93f569c7a904",
      "content-length": "123",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### 상호 작용 데이터 세트 가져오기 작업이 완료될 때까지 기다립니다.

다음 셀은 두 가져오기 작업이 완료될 때까지 기다립니다.

In [39]:
%%time

in_progress_import_arns = [ dataset_import_job_without_ints_arn, dataset_import_job_with_ints_arn ]

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    for import_arn in in_progress_import_arns:
        describe_dataset_import_job_response = personalize.describe_dataset_import_job(
            datasetImportJobArn = import_arn
        )
        status = describe_dataset_import_job_response["datasetImportJob"]['status']
        if status == "ACTIVE":
            print("Dataset import succeeded for {}".format(import_arn))
            in_progress_import_arns.remove(import_arn)
        elif status == "CREATE FAILED":
            print("Create failed for {}".format(import_arn))
            in_progress_import_arns.remove(import_arn)

    if len(in_progress_import_arns) <= 0:
        break
    else:
        print("At least one dataset import job is still in progress")
                
    time.sleep(60)

At least one dataset import job is still in progress
At least one dataset import job is still in progress
At least one dataset import job is still in progress
At least one dataset import job is still in progress
Dataset import succeeded for arn:aws:personalize:us-east-1:224124347618:dataset-import-job/amazon-pantry-without-desc-ints-import
At least one dataset import job is still in progress
Dataset import succeeded for arn:aws:personalize:us-east-1:224124347618:dataset-import-job/amazon-pantry-with-desc-ints-import
CPU times: user 42.3 ms, sys: 6.1 ms, total: 48.4 ms
Wall time: 5min


### 항목 데이터 세트 스키마 및 데이터 세트 생성

다음으로 항목 데이터 세트에 대해 이 프로세스를 반복합니다. 하지만 이번에는 항목 데이터 세트 하나에는 설명 열이 포함되어 있고 다른 항목 데이터 세트에는 포함되어 있지 않으므로 스키마를 2개 만들어야 합니다. 설명을 포함하지 않는 스키마부터 시작하겠습니다.

In [40]:
item_without_desc_schema = {
    "type": "record",
    "name": "Items",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "BRAND",
            "type": [ "null", "string" ],
            "categorical": True
        },{
            "name": "PRICE",
            "type": [ "null", "float" ],
        }
    ],
    "version": "1.0"
}

create_schema_response = personalize.create_schema(
    name = "amazon-pantry-item-without-desc-schema",
    schema = json.dumps(item_without_desc_schema)
)

item_without_desc_schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

{
  "schemaArn": "arn:aws:personalize:us-east-1:224124347618:schema/amazon-pantry-item-without-desc-schema",
  "ResponseMetadata": {
    "RequestId": "11a18b05-189b-4485-ba8b-58083e784321",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 13 Jul 2021 22:17:16 GMT",
      "x-amzn-requestid": "11a18b05-189b-4485-ba8b-58083e784321",
      "content-length": "104",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


다음으로, 설명이 포함된 스키마를 만들겠습니다. `DESCRIPTION` 필드에 `"textual": True` 속성이 있는 것을 주목하세요. 비정형 텍스트 필드와 범주형 및 문자열 필드는 이렇게 구분합니다. 이 속성이 없으면 Personalize는 이 텍스트에서 특성을 추출할 때 자연어 처리 기술을 적용하지 않습니다.

In [41]:
item_with_desc_schema = {
    "type": "record",
    "name": "Items",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "BRAND",
            "type": [ "null", "string" ],
            "categorical": True
        },{
            "name": "PRICE",
            "type": [ "null", "float" ],
        },{
            "name": "DESCRIPTION",
            "type": [ "null", "string" ],
            "textual": True
        }
    ],
    "version": "1.0"
}

create_schema_response = personalize.create_schema(
    name = "amazon-pantry-item-with-desc-schema",
    schema = json.dumps(item_with_desc_schema)
)

item_with_desc_schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

{
  "schemaArn": "arn:aws:personalize:us-east-1:224124347618:schema/amazon-pantry-item-with-desc-schema",
  "ResponseMetadata": {
    "RequestId": "05c60e80-1d7c-45f1-a881-d08af62a8432",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 13 Jul 2021 22:17:16 GMT",
      "x-amzn-requestid": "05c60e80-1d7c-45f1-a881-d08af62a8432",
      "content-length": "101",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


다음으로 각 데이터 세트 그룹에 Personalize 데이터 세트를 만듭니다. 이때 각 데이터 세트에 대한 적절한 스키마 ARN을 지정하도록 주의합니다.

In [42]:
dataset_type = "ITEMS"
create_dataset_response = personalize.create_dataset(
    name = "amazon-pantry-without-desc-items",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_without_desc_arn,
    schemaArn = item_without_desc_schema_arn
)

items_dataset_without_desc_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

{
  "datasetArn": "arn:aws:personalize:us-east-1:224124347618:dataset/amazon-pantry-without-desc/ITEMS",
  "ResponseMetadata": {
    "RequestId": "f9b87f6c-22c8-42a7-8a3a-8c203300e3eb",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 13 Jul 2021 22:18:13 GMT",
      "x-amzn-requestid": "f9b87f6c-22c8-42a7-8a3a-8c203300e3eb",
      "content-length": "100",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [43]:
create_dataset_response = personalize.create_dataset(
    name = "amazon-pantry-with-desc-items",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_with_desc_arn,
    schemaArn = item_with_desc_schema_arn
)

items_dataset_with_desc_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

{
  "datasetArn": "arn:aws:personalize:us-east-1:224124347618:dataset/amazon-pantry-with-desc/ITEMS",
  "ResponseMetadata": {
    "RequestId": "cbba4d2a-17c9-4a9b-a6a3-a2df934e2de1",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 13 Jul 2021 22:18:16 GMT",
      "x-amzn-requestid": "cbba4d2a-17c9-4a9b-a6a3-a2df934e2de1",
      "content-length": "97",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


#### S3에 항목 CSV 스테이징

다음으로, 위에서 만든 것과 동일한 S3 버킷에 2개의 항목 CSV 파일을 복사합니다.

In [44]:
boto3.Session().resource('s3').Bucket(bucket_name).Object(items_without_desc_filename).upload_file(items_without_desc_filename)

In [45]:
boto3.Session().resource('s3').Bucket(bucket_name).Object(items_with_desc_filename).upload_file(items_with_desc_filename)

### 각 데이터 세트 그룹의 항목 데이터 세트 가져오기

S3 버킷 정책 및 IAM 역할이 이미 설정되어 있으므로, 2개의 데이터 세트 가져오기 작업만 제출하면 항목 CSV를 가져올 수 있습니다.

In [46]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "amazon-pantry-without-desc-items-import",
    datasetArn = items_dataset_without_desc_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, items_without_desc_filename)
    },
    roleArn = role_arn
)

dataset_import_job_without_items_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

{
  "datasetImportJobArn": "arn:aws:personalize:us-east-1:224124347618:dataset-import-job/amazon-pantry-without-desc-items-import",
  "ResponseMetadata": {
    "RequestId": "c6ea207b-b8eb-4565-8ce2-6eb44e0fa18f",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 13 Jul 2021 22:18:37 GMT",
      "x-amzn-requestid": "c6ea207b-b8eb-4565-8ce2-6eb44e0fa18f",
      "content-length": "127",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [47]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "amazon-pantry-with-desc-items-import",
    datasetArn = items_dataset_with_desc_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, items_with_desc_filename)
    },
    roleArn = role_arn
)

dataset_import_job_with_items_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

{
  "datasetImportJobArn": "arn:aws:personalize:us-east-1:224124347618:dataset-import-job/amazon-pantry-with-desc-items-import",
  "ResponseMetadata": {
    "RequestId": "cf95206f-1527-4247-a618-6c8c832fa05f",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 13 Jul 2021 22:18:38 GMT",
      "x-amzn-requestid": "cf95206f-1527-4247-a618-6c8c832fa05f",
      "content-length": "124",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### 항목 가져오기 작업이 완료될 때까지 기다리기

다음 로직은 두 항목 데이터 세트를 모두 각각의 데이터 세트 그룹으로 완전히 가져올 때까지 기다립니다.

In [48]:
%%time

in_progress_import_arns = [ dataset_import_job_without_items_arn, dataset_import_job_with_items_arn ]

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    for import_arn in in_progress_import_arns:
        describe_dataset_import_job_response = personalize.describe_dataset_import_job(
            datasetImportJobArn = import_arn
        )
        status = describe_dataset_import_job_response["datasetImportJob"]['status']
        if status == "ACTIVE":
            print("Dataset import succeeded for {}".format(import_arn))
            in_progress_import_arns.remove(import_arn)
        elif status == "CREATE FAILED":
            print("Create failed for {}".format(import_arn))
            in_progress_import_arns.remove(import_arn)

    if len(in_progress_import_arns) <= 0:
        break
    else:
        print("At least one dataset import job is still in progress")
                
    time.sleep(60)

At least one dataset import job is still in progress
At least one dataset import job is still in progress
At least one dataset import job is still in progress
At least one dataset import job is still in progress
Dataset import succeeded for arn:aws:personalize:us-east-1:224124347618:dataset-import-job/amazon-pantry-with-desc-items-import
At least one dataset import job is still in progress
At least one dataset import job is still in progress
At least one dataset import job is still in progress
Dataset import succeeded for arn:aws:personalize:us-east-1:224124347618:dataset-import-job/amazon-pantry-without-desc-items-import
CPU times: user 57.6 ms, sys: 5.06 ms, total: 62.7 ms
Wall time: 7min


## 솔루션 및 솔루션 버전 생성

상호 작용 및 항목 데이터 세트를 각 데이터 세트 그룹으로 가져왔으므로, 이제 사용자-개인화 레시피를 사용하여 각 데이터 세트 그룹의 데이터에 대한 솔루션과 솔루션 버전을 만듭니다.

먼저 사용 가능한 Personalize 레시피를 나열해 보겠습니다.

In [49]:
personalize.list_recipes()

{'recipes': [{'name': 'aws-hrnn',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-hrnn',
   'status': 'ACTIVE',
   'creationDateTime': datetime.datetime(2019, 6, 10, 0, 0, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(2021, 2, 6, 19, 6, 40, 447000, tzinfo=tzlocal())},
  {'name': 'aws-hrnn-coldstart',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-hrnn-coldstart',
   'status': 'ACTIVE',
   'creationDateTime': datetime.datetime(2019, 6, 10, 0, 0, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(2021, 2, 6, 19, 6, 40, 447000, tzinfo=tzlocal())},
  {'name': 'aws-hrnn-metadata',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-hrnn-metadata',
   'status': 'ACTIVE',
   'creationDateTime': datetime.datetime(2019, 6, 10, 0, 0, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(2021, 2, 6, 19, 6, 40, 447000, tzinfo=tzlocal())},
  {'name': 'aws-personalized-ranking',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-personalized-ranking',
   'stat

이 노트북에는 항목 메타데이터를 사용하는 레시피 중 하나인 사용자 개인화 레시피를 사용합니다. 이 레시피는 표준 개인화 사용 사례를 지원하며, 사용자가 관심을 가질 만한 항목을 Personalize가 추천하도록 해야 합니다. 

In [50]:
user_personalization_recipe_arn = "arn:aws:personalize:::recipe/aws-user-personalization"

먼저 데이터 세트 그룹에 항목 설명이 포함되지 않은 솔루션과 솔루션 버전을 만들겠습니다.

In [51]:
user_personalization_create_solution_response = personalize.create_solution(
    name = "amazon-pantry-without-desc-userpersonalization",
    datasetGroupArn = dataset_group_without_desc_arn,
    recipeArn = user_personalization_recipe_arn
)

user_personalization_without_desc_solution_arn = user_personalization_create_solution_response['solutionArn']

In [52]:
print(user_personalization_without_desc_solution_arn)

arn:aws:personalize:us-east-1:224124347618:solution/amazon-pantry-without-desc-userpersonalization


In [53]:
user_personalization_solution_version_response = personalize.create_solution_version(
    solutionArn = user_personalization_without_desc_solution_arn
)

In [54]:
user_personalization_without_solution_version_arn = user_personalization_solution_version_response['solutionVersionArn']
print(json.dumps(user_personalization_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:224124347618:solution/amazon-pantry-without-desc-userpersonalization/0b76212f",
  "ResponseMetadata": {
    "RequestId": "018b3fcb-10d5-4290-a17a-3970723abacd",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 13 Jul 2021 22:26:14 GMT",
      "x-amzn-requestid": "018b3fcb-10d5-4290-a17a-3970723abacd",
      "content-length": "132",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


다음으로, 데이터 세트 그룹에 항목 설명이 포함된 솔루션과 솔루션 버전을 만들겠습니다.

In [55]:
user_personalization_create_solution_response = personalize.create_solution(
    name = "amazon-pantry-with-desc-userpersonalization",
    datasetGroupArn = dataset_group_with_desc_arn,
    recipeArn = user_personalization_recipe_arn
)

user_personalization_with_desc_solution_arn = user_personalization_create_solution_response['solutionArn']

In [56]:
print(user_personalization_with_desc_solution_arn)

arn:aws:personalize:us-east-1:224124347618:solution/amazon-pantry-with-desc-userpersonalization


In [57]:
user_personalization_solution_version_response = personalize.create_solution_version(
    solutionArn = user_personalization_with_desc_solution_arn
)

In [58]:
user_personalization_with_solution_version_arn = user_personalization_solution_version_response['solutionVersionArn']
print(json.dumps(user_personalization_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:224124347618:solution/amazon-pantry-with-desc-userpersonalization/f178990f",
  "ResponseMetadata": {
    "RequestId": "f630b862-6fa9-4eb7-a0d2-71d0b9637e80",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 13 Jul 2021 22:26:29 GMT",
      "x-amzn-requestid": "f630b862-6fa9-4eb7-a0d2-71d0b9637e80",
      "content-length": "129",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### 솔루션 버전이 활성 상태가 될 때까지 기다리기

마지막으로 솔루션 버전 생성이 완료될 때까지 기다립니다. 이 단계는 Personalize가 데이터 세트와 선택한 레시피를 기반으로 기계 학습 모델을 훈련하는 단계입니다. 또한 Personalize는 보류된 데이터를 사용하여 훈련된 모델의 추천 품질을 평가할 수 있도록 상호 작용 데이터 세트를 훈련 부분과 평가 부분으로 분할합니다.

데이터 세트 그룹의 솔루션 버전 중 설명 데이터가 포함된 버전은 설명이 없는 버전보다 훈련하는 데 더 오래 걸립니다.

In [59]:
%%time

in_progress_solution_versions = [
    user_personalization_without_solution_version_arn,
    user_personalization_with_solution_version_arn
]

max_time = time.time() + 10*60*60 # 10 hours
while time.time() < max_time:
    for solution_version_arn in in_progress_solution_versions:
        version_response = personalize.describe_solution_version(
            solutionVersionArn = solution_version_arn
        )
        status = version_response["solutionVersion"]["status"]
        
        if status == "ACTIVE":
            print("Build succeeded for {}".format(solution_version_arn))
            in_progress_solution_versions.remove(solution_version_arn)
        elif status == "CREATE FAILED":
            print("Build failed for {}".format(solution_version_arn))
            in_progress_solution_versions.remove(solution_version_arn)
    
    if len(in_progress_solution_versions) <= 0:
        break
    else:
        print("At least one solution build is still in progress")
        
    time.sleep(60)

At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solutio

일반적으로 텍스트 기반 비정형 메타데이터를 추가하면 훈련 시간이 늘어납니다. 이 예에서는 제품 설명이 포함된 데이터 세트로 훈련된 솔루션 버전이 제품 설명이 없는 데이터 세트로 훈련된 솔루션 버전보다 약 15분 더 오래 걸렸다는 것을 알 수 있습니다. 이 차이는 데이터 세트의 구성과 텍스트 값에 따라 달라집니다.

솔루션 버전별 훈련 시간을 살펴보고 비교해보겠습니다.

In [60]:
response = personalize.describe_solution_version(solutionVersionArn = user_personalization_without_solution_version_arn)
training_hours_without_desc = response['solutionVersion']['trainingHours']

response = personalize.describe_solution_version(solutionVersionArn = user_personalization_with_solution_version_arn)
training_hours_with_desc = response['solutionVersion']['trainingHours']
training_diff = (training_hours_with_desc - training_hours_without_desc) / training_hours_without_desc

print(f"Training hours without description: {training_hours_without_desc}")
print(f"Training hours with description: {training_hours_with_desc}")

print("Difference of {:.2%}".format(training_diff))

Training hours without description: 4.199
Training hours with description: 5.346
Difference of 27.32%


비용 계산에 사용된 훈련 시간은 설명 열을 사용한 훈련에서 약 27% 더 길었습니다.

Wall Clock Time(WCT) 및 훈련 시간은 데이터 세트의 크기에 따라 다르지만, 이 정보는 데이터 세트에 비정형 텍스트를 추가하는 것을 고려하는 경우에 균형을 평가하는 데 도움이 될 수 있습니다.

### 오프라인 지표 살펴보기

이제 솔루션 버전 구축을 마쳤으므로 각 솔루션 버전에 대한 오프라인 지표를 살펴보고 비교하여 비정형 텍스트가 이러한 지표에 어떻게 영향을 미쳤는지 알아보겠습니다.

In [61]:
metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = user_personalization_without_solution_version_arn
)

print(json.dumps(metrics_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:224124347618:solution/amazon-pantry-without-desc-userpersonalization/0b76212f",
  "metrics": {
    "coverage": 0.0914,
    "mean_reciprocal_rank_at_25": 0.0268,
    "normalized_discounted_cumulative_gain_at_10": 0.0376,
    "normalized_discounted_cumulative_gain_at_25": 0.0464,
    "normalized_discounted_cumulative_gain_at_5": 0.0309,
    "precision_at_10": 0.0058,
    "precision_at_25": 0.0037,
    "precision_at_5": 0.0076
  },
  "ResponseMetadata": {
    "RequestId": "8c61339a-f929-47e0-81f0-a9660ebd589f",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 13 Jul 2021 23:16:07 GMT",
      "x-amzn-requestid": "8c61339a-f929-47e0-81f0-a9660ebd589f",
      "content-length": "430",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


두 솔루션 버전 간에 지표를 더 쉽게 비교할 수 있도록 이들 지표를 사전에 저장해 두겠습니다.

In [62]:
metrics = {
    'Coverage': [ metrics_response['metrics']['coverage'] ],
    'MRR-25': [ metrics_response['metrics']['mean_reciprocal_rank_at_25'] ],
    'NDCG-5': [ metrics_response['metrics']['normalized_discounted_cumulative_gain_at_5'] ],
    'NDCG-10': [ metrics_response['metrics']['normalized_discounted_cumulative_gain_at_10'] ],
    'NDCG-25': [ metrics_response['metrics']['normalized_discounted_cumulative_gain_at_25'] ],    
    'Precision-5': [ metrics_response['metrics']['precision_at_5'] ],
    'Precision-10': [ metrics_response['metrics']['precision_at_10'] ],
    'Precision-25': [ metrics_response['metrics']['precision_at_25'] ],    
}

다음으로, 설명 열이 포함된 솔루션 버전에 대한 오프라인 지표를 가져와 저장합니다.

In [63]:
metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = user_personalization_with_solution_version_arn
)

print(json.dumps(metrics_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:224124347618:solution/amazon-pantry-with-desc-userpersonalization/f178990f",
  "metrics": {
    "coverage": 0.1323,
    "mean_reciprocal_rank_at_25": 0.0367,
    "normalized_discounted_cumulative_gain_at_10": 0.049,
    "normalized_discounted_cumulative_gain_at_25": 0.0591,
    "normalized_discounted_cumulative_gain_at_5": 0.0425,
    "precision_at_10": 0.0071,
    "precision_at_25": 0.0045,
    "precision_at_5": 0.0104
  },
  "ResponseMetadata": {
    "RequestId": "b54df693-4378-4194-96c7-cec3a9d934cf",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 13 Jul 2021 23:16:14 GMT",
      "x-amzn-requestid": "b54df693-4378-4194-96c7-cec3a9d934cf",
      "content-length": "426",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [64]:
metrics['Coverage'].append(metrics_response['metrics']['coverage'])
metrics['MRR-25'].append(metrics_response['metrics']['mean_reciprocal_rank_at_25'])
metrics['NDCG-5'].append(metrics_response['metrics']['normalized_discounted_cumulative_gain_at_5'])
metrics['NDCG-10'].append(metrics_response['metrics']['normalized_discounted_cumulative_gain_at_10'])
metrics['NDCG-25'].append(metrics_response['metrics']['normalized_discounted_cumulative_gain_at_25'])
metrics['Precision-5'].append(metrics_response['metrics']['precision_at_5'])
metrics['Precision-10'].append(metrics_response['metrics']['precision_at_10'])
metrics['Precision-25'].append(metrics_response['metrics']['precision_at_25'])

텍스트가 있는 것과 없는 것 각각의 지표 변화율 계산하고 결과를 표시합니다.

In [65]:
for key in metrics:
    metrics[key].append("{:.2%}".format((metrics[key][1] - metrics[key][0])/metrics[key][0]))

metrics_df = pd.DataFrame.from_dict(metrics,orient='index',columns=['Without Text', 'With Text', '% Change'])
metrics_df

,Without Text,With Text,% Change
Coverage,0.0914,0.1323,44.75%
MRR-25,0.0268,0.0367,36.94%
NDCG-5,0.0309,0.0425,37.54%
NDCG-10,0.0376,0.0490,30.32%
NDCG-25,0.0464,0.0591,27.37%
Precision-5,0.0076,0.0104,36.84%
Precision-10,0.0058,0.0071,22.41%
Precision-25,0.0037,0.0045,21.62%


이러한 지표를 통해 항목 설명이 포함된 솔루션 버전의 추천이 전반적으로 훨씬 더 우수하다는 것을 알 수 있습니다. 사용자와 항목의 상호 작용이 적은 희박한 상호 작용 데이터 세트의 경우, 항목 및/또는 사용자당 상호 작용 수가 더 많은 데이터 세트보다 텍스트를 추가할 때의 이점이 더 큽니다.

## 정리

이 노트북에서 생성한 Personalize 리소스는 AWS 콘솔의 Personalize 서비스 페이지에서 삭제할 수 있습니다.

또는 다음 스크립트를 로컬로 실행하여 각 데이터 세트 그룹의 리소스를 모두 삭제할 수 있습니다.

https://gist.github.com/james-jory/62ddddf2f9180b77dd2a42e645b9d3b0

또한 IAM 역할과 S3 버킷은 IAM 및 S3 서비스 페이지에서 각각 삭제할 수 있습니다.